In [83]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, Model # type: ignore
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense, LSTM, Bidirectional, Input, RepeatVector, Concatenate # type: ignore
from sklearn.model_selection import train_test_split
from functools import partial
from tqdm import tqdm   # type: ignore
import math

In [2]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [18]:

train_df['event_id'] = train_df['event_id'].apply(lambda x: '_'.join(x.split('_')[0:2]))
train_df['event_idx'] = train_df.groupby('event_id', sort=False).ngroup()
test_df['event_id'] = test_df['event_id'].apply(lambda x: '_'.join(x.split('_')[0:2]))
test_df['event_idx'] = test_df.groupby('event_id', sort=False).ngroup()

train_df['event_t'] = train_df.groupby('event_id').cumcount()
test_df['event_t'] = test_df.groupby('event_id').cumcount()

print(train_df.head())
print(test_df.head())

          event_id  precipitation  label  event_idx  event_t
0  id_spictby0jfsb       0.000000      0          0        0
1  id_spictby0jfsb       0.095438      0          0        1
2  id_spictby0jfsb       1.949560      0          0        2
3  id_spictby0jfsb       3.232160      0          0        3
4  id_spictby0jfsb       0.000000      0          0        4
          event_id  precipitation  event_idx  event_t
0  id_j7b6sokflo4k        0.00000          0        0
1  id_j7b6sokflo4k        3.01864          0        1
2  id_j7b6sokflo4k        0.00000          0        2
3  id_j7b6sokflo4k       16.61520          0        3
4  id_j7b6sokflo4k        2.56706          0        4


In [8]:
BAND_NAMES = ('B2', 'B3','B4', 'B8', 'B11', 'slope')
H, W, NUM_CHANNELS = IMG_DIM= (128, 128, len(BAND_NAMES))
_MAX_INT = np.iinfo(np.int16).max


In [11]:
def decode_slope(X: np.ndarray) -> np.ndarray:
    return (X / _MAX_INT * (math.pi / 2.0 )).astype(np.float32)

In [12]:
def normalize(x: np.ndarray, mean: int, std: int) -> np.ndarray:
    return (x - mean) / std


In [13]:
rough_S2_normalize = partial(normalize, mean=1250, std=500)

In [14]:
def preprocess_image(x: np.ndarray) -> np.ndarray:
    return np.concatenate([
        rough_S2_normalize(x[..., :-1].astype(np.float32)),
        decode_slope(x[..., -1:]),
    ], axis=-1, dtype=np.float32)


In [16]:
composite_images = np.load('composite_images.npz')
images_path = 'composite_images.npz'

In [15]:
def preprocess_data_and_images(data_df, composite_images):
    event_ids = data_df['event_id'].unique()
    timeseries = []
    labels = []
    images = []

    for event_id in tqdm(event_ids, desc="Processing data"):
        event_data = data_df[data_df['event_id'] == event_id]
        timeseries.append(event_data['precipitation'].values)  # Shape: (730,)
        if 'label' in event_data.columns:
            labels.append(event_data['label'].values)  # Shape: (730,)
        images.append(preprocess_image(composite_images[event_id]))  # Shape: (128, 128, 6)

    timeseries = np.array(timeseries)
    labels = np.array(labels) if labels else None
    images = np.stack(images, axis=0)

    return timeseries, labels, images

In [19]:
train_timeseries, train_labels, train_images = preprocess_data_and_images(train_df, composite_images)


Processing data: 100%|██████████| 674/674 [01:42<00:00,  6.55it/s]


In [54]:
# Split into training and validation sets
train_split, val_split = train_test_split(
    np.arange(len(train_timeseries)), test_size=0.2, random_state=42
)

X_precip_train, X_precip_val = train_timeseries[train_split], train_timeseries[val_split]
y_train, y_val = train_labels[train_split], train_labels[val_split]
X_img_train, X_img_val = train_images[train_split], train_images[val_split]

In [39]:
from sklearn.utils.class_weight import compute_class_weight
# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y_train.argmax(axis=1)), y=y_train.argmax(axis=1))
class_weight_dict = {i: weight for i, weight in enumerate(class_weights)}

In [84]:
# Build the custom neural network
# Image encoder
image_input = Input(shape=(128, 128, 6), name='image_input')
x = Conv2D(32, (3, 3), activation='relu', padding='same')(image_input)
x = MaxPooling2D((2, 2))(x)
x = Dropout(0.2)(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
encoded_image = Dense(128, activation='relu')(x)

In [85]:
# Time-series input
precip_input = Input(shape=(730,), name='precip_input')

# Custom layer to wrap tf.expand_dims
class ExpandDimsLayer(layers.Layer):
	def call(self, inputs):
		return tf.expand_dims(inputs, axis=-1)

# Repeat the image encoding vector 730 times and concatenate with precipitation data
repeated_image_vector = RepeatVector(730)(encoded_image)
expanded_precip_input = ExpandDimsLayer()(precip_input)
concatenated = Concatenate(axis=-1)([repeated_image_vector, expanded_precip_input])


In [86]:
# Bidirectional LSTM
x = Bidirectional(LSTM(64, return_sequences=True))(concatenated)
x = Dense(64, activation='relu')(x)
day_probabilities = Dense(1, activation='sigmoid')(x)


In [87]:
# Define the model
model = Model(inputs=[image_input, precip_input], outputs=day_probabilities)


In [88]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [89]:
# Early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=3, restore_best_weights=True
)

In [103]:
# Compute class weights
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels.argmax(axis=1)),
    y=train_labels.argmax(axis=1)
)
class_weight_dict = {i: weight for i, weight in enumerate(class_weights)}

In [96]:
from tensorflow.keras.losses import binary_crossentropy #type: ignore


def focal_loss(gamma=2., alpha=0.25):
    def loss(y_true, y_pred):
        bce = binary_crossentropy(y_true, y_pred)
        pt = tf.exp(-bce)  # Probabilities of predictions
        focal = alpha * (1 - pt) ** gamma * bce
        return tf.reduce_mean(focal)
    return loss

# Update model compilation
model.compile(optimizer='adam', loss=focal_loss(gamma=2., alpha=0.25), metrics=['accuracy'])


In [ ]:
model.fit(
    [X_img_train, X_precip_train], y_train,
    validation_data=([X_img_val, X_precip_val], y_val),
    epochs=50,
    batch_size=32,
    callbacks=[early_stopping], 
    class_weight=class_weight_dict  
)


Epoch 1/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 55s 1s/step - accuracy: 0.9993 - loss: 3.9799e-08 - val_accuracy: 0.9995 - val_loss: 6.9690e-08
Epoch 2/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - accuracy: 0.9994 - loss: 3.5461e-08 - val_accuracy: 0.9995 - val_loss: 6.9663e-08
Epoch 3/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 22s 1s/step - accuracy: 0.9993 - loss: 3.5290e-08 - val_accuracy: 0.9995 - val_loss: 6.9610e-08
Epoch 4/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - accuracy: 0.9993 - loss: 3.7521e-08 - val_accuracy: 0.9995 - val_loss: 6.9539e-08
Epoch 5/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 22s 1s/step - accuracy: 0.9993 - loss: 4.1164e-08 - val_accuracy: 0.9995 - val_loss: 6.9485e-08
Epoch 6/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 27s 2s/step - accuracy: 0.9993 - loss: 4.1784e-08 - val_accuracy: 0.9995 - val_loss: 6.9622e-08
Epoch 7/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 25s 1s/step - accuracy: 0.9994 - loss: 3.7289e-08 - val_accuracy: 0.9995 - val_loss: 6.9574e-08
Epoch 8/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 26s 2s/step - accuracy: 

In [30]:
model.save('flood_prediction_model.keras')

In [105]:
# Preprocess test data
test_timeseries, _, test_images = preprocess_data_and_images(test_df, composite_images)


Processing data: 100%|██████████| 224/224 [00:09<00:00, 22.56it/s]


In [106]:
predictions = model.predict([test_images, test_timeseries])

7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 452ms/step


In [107]:
sample_submission = pd.read_csv('SampleSubmission (2).csv')
sample_submission.head()

,event_id,label
0,id_j7b6sokflo4k_X_0,0
1,id_j7b6sokflo4k_X_1,0
2,id_j7b6sokflo4k_X_2,0
3,id_j7b6sokflo4k_X_3,0
4,id_j7b6sokflo4k_X_4,0


In [111]:
sample_submission['label'] = predictions.flatten()
sample_submission.head()

,event_id,label
0,id_j7b6sokflo4k_X_0,0.003741
1,id_j7b6sokflo4k_X_1,0.002437
2,id_j7b6sokflo4k_X_2,0.002437
3,id_j7b6sokflo4k_X_3,0.003076
4,id_j7b6sokflo4k_X_4,0.001912


In [110]:
sample_submission.to_csv("submission_409.csv", index=False)
